# Elbow data analysis

In [30]:
# analyze_elbow_effusion.py
import pandas as pd
from pathlib import Path
import numpy as np
import os
import re
import nibabel as nib


## Elbow Ultrasound Multireader Trial

- This notebook processes and analyzes elbow ultrasound data from a multireader trial.
- The dataset contains fracture and effusion annotations from two radiologists: JJ and JK.
- For consistency, we limit our analysis to labels from **JJ** only.


Reference spreadsheet: [Google Sheets](https://docs.google.com/spreadsheets/d/14an9mxel6_QW3PvJL-gBSHTyw-nTkD2m/edit?usp=sharing&ouid=101373979316677349346&rtpof=true&sd=true)


In [31]:
# load and select JJ’s columns
csv_path = Path(r"M:\ElbowProject\Elbow XR US.csv")   
df = pd.read_csv(csv_path)
df

,Randomized ID,Study ID,JK US Fracture,JJ US Fracture,US Fracture Agree?,JK XR Fracture,JJ XR Fracture,XR Fracture Agree?,JK US Effusion?,JJ US Effusion?,US Effusion Agree?,JK XR Effusion?,JJ XR Effusion?,XR Effusion Agree?
0,EL001,88105 elbow 2D,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0
1,EL002,88098 elbow 2D,1,1,2,1.0,1.0,2,1,1,2,1.0,1.0,2
2,EL003,W00004 elbow 3D,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0
3,EL004,88106 elbow 2D,0,0,0,0.0,0.0,0,1,1,2,0.0,0.0,0
4,EL005,88026 elbow 2D,0,1,1,0.0,0.0,0,1,1,2,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,EL128,88099 elbow 3D,0,0,0,1.0,0.0,1,1,1,2,1.0,1.0,2
128,EL129,88070 elbow 3D,0,0,0,NaN,NaN,0,0,0,0,NaN,NaN,0
129,EL130,88162 elbow 3D,0,0,0,0.0,0.0,0,1,0,1,0.0,0.0,0
130,EL131,88055 elbow 3D,0,0,0,0.0,0.0,0,0,0,0,0.0,0.0,0


In [32]:
all_rows = len(df)
df.columns

Index(['Randomized ID', 'Study ID', 'JK US Fracture', 'JJ US Fracture',
       'US Fracture Agree?', 'JK XR Fracture', 'JJ XR Fracture',
       'XR Fracture Agree?', 'JK US Effusion?', 'JJ US Effusion?',
       'US Effusion Agree?', 'JK XR Effusion?', 'JJ XR Effusion?',
       'XR Effusion Agree?'],
      dtype='object')

In [33]:
# Load and Clean JJ-Labeled Data
# We rename relevant JJ columns and drop rows with missing values in the diagnostic labels.


keep = {
    "Study ID"         : "study_id",
    "JJ US Effusion?"  : "us_effusion",
    "JJ XR Effusion?"  : "xr_effusion",
    "JJ US Fracture"   : "us_fracture",
    "JJ XR Fracture"   : "xr_fracture",
}

df = df[keep.keys()].rename(columns=keep)

# force 0/1 integers, drop rows with NaNs in diagnostic columns
diag = ["us_effusion", "xr_effusion", "us_fracture", "xr_fracture"]
df[diag] = df[diag].apply(pd.to_numeric, errors="coerce").astype("Int64")
# Identify rows that will be dropped (have NaN in any of the diag columns)
dropped_rows = df[df[diag].isnull().any(axis=1)]
# Summary of Dropped Rows
# Print their study IDs
print("Dropped study IDs due to missing diagnostic labels:")
dropped_studies = dropped_rows['study_id'].tolist()
print(dropped_studies)

df = df.dropna(subset=diag).astype({c: int for c in diag})

print(f"Dataset ready: {len(df)} examinations")

Dropped study IDs due to missing diagnostic labels:
['W00004 elbow 3D', '88137 elbow 3D', '88128 elbow 2D', '88126 elbow 2D', '88137 elbow 2D', '88159 elbow 2D', 'W00001 elbow 3D', '88019 elbow 3D', '88102 elbow 3D', 'W00008 elbow 2D', '88019 elbow 2D', '88061 elbow 3D', 'W00008 elbow 3D', '88061 elbow 2D', '88159 elbow 3D', '88128 elbow 3D', '88057 elbow 3D', '88102 elbow 2D', '88028 elbow 2D', '88070 elbow 2D', '88057 elbow 2D', 'W00001 elbow 2D', '88070 elbow 3D']
Dataset ready: 109 examinations


In [34]:
print(f"Dataset has {len(dropped_studies)} rows with missing information")
print(f"Cleaned dataset has {len(df)} rows")

Dataset has 23 rows with missing information
Cleaned dataset has 109 rows


- A total of 23 studies were excluded due to missing fracture or effusion labels from radiologist JJ. This leaves us with 109 fully labeled studies for analysis.

### Elbow Ultrasound Study Types

This spreadsheet includes both **2D** and **3D** ultrasound studies:

- **2D studies** were acquired using a **Philips Lumify** system with an **L5-12 (5–12 MHz) transducer**.
- **3D studies** were acquired using a **Philips IU22** system with a **VL13-5 (13 MHz) transducer**.


In [35]:
# Categorize dropped studies based on whether they belong to 2D (Lumify) or 3D (IU22) acquisition.
# Extract dropped ID and categorize
IDs_2D = []
IDs_3D = []

for s in dropped_studies:
    match = re.search(r'(W\d{5}|\d{5})', s)
    if match:
        study_id = match.group()
        if '2D' in s:
            IDs_2D.append(study_id)
        elif '3D' in s:
            IDs_3D.append(study_id)

# Results
print("Dropped 2D IDs:", IDs_2D)
print("Dropped 3D IDs:", IDs_3D)

Dropped 2D IDs: ['88128', '88126', '88137', '88159', 'W00008', '88019', '88061', '88102', '88028', '88070', '88057', 'W00001']
Dropped 3D IDs: ['W00004', '88137', 'W00001', '88019', '88102', '88061', 'W00008', '88159', '88128', '88057', '88070']


- Among the dropped cases, 12 were 2D studies (from Lumify) and 11 were 3D studies (from IU22). These are excluded from further analysis due to incomplete annotations.

### Extracting and Summarizing Study IDs by Modality

In this step, we:

1. **Extracted numeric study IDs** from the `study_id` field.
2. **Grouped studies** based on imaging modality:
   - **2D** studies (Philips Lumify L5-12 MHz)
   - **3D** studies (Philips IU22 VL13-5 MHz)
3. **Counted total and unique studies** to ensure proper tracking and avoid duplicate entries.

In [9]:
# === Extract alphanumeric file IDs from 'study_id' ===
df['file_id'] = df['study_id'].str.extract(r'(W\d{5}|\d{5})', expand=False).str.upper()

# === Categorize as 2D or 3D ===
IDs_2D = df[df['study_id'].str.contains('2D', case=False)]['file_id'].tolist()
IDs_3D = df[df['study_id'].str.contains('3D', case=False)]['file_id'].tolist()
file_IDs = df['file_id'].tolist()

# === Print summary ===
print("file_IDs =", file_IDs)
print("file_IDs: ", len(file_IDs))
print("IDs_2D =", IDs_2D)
print("IDs_2D: ", len(IDs_2D))
print("IDs_3D =", IDs_3D)
print("IDs_3D: ", len(IDs_3D))

# Unique IDs
unique_ids = np.unique(file_IDs)
print("unique studies:", unique_ids)
print("Number of unique studies:", len(unique_ids))

# === Differences ===

# === Compute differences ===

set_2D = set(IDs_2D)
set_3D = set(IDs_3D)

only_in_2D = sorted(set_2D - set_3D)
only_in_3D = sorted(set_3D - set_2D)

print("Present in 2D but not in 3D:", only_in_2D)
print("Count:", len(only_in_2D))

print("Present in 3D but not in 2D:", only_in_3D)
print("Count:", len(only_in_3D))


file_IDs = ['88105', '88098', '88106', '88026', '88042', '88098', '88148', '88023', '88068', '88022', '88025', '88072', '88105', '88133', '88156', '88077', '88069', '88141', '88093', '88017', '88118', '88131', '88132', '88158', '88082', '88074', '88145', '88034', '88095', '88133', '88015', '88077', '88082', '88145', '88135', '88112', '88131', '88118', '88069', '88170', '88010', '88090', '88138', '88114', '88091', '88150', '88015', '88072', '88075', '88036', '88034', '88112', '88111', '88037', '88091', '88010', '88152', '88150', '88058', '88037', '88078', '88096', '88170', '88094', '88119', '88017', '88172', '88154', '88025', '88148', '88135', '88094', '88114', '88154', '88084', '88078', '88099', '88033', '88111', '88132', '88084', '88095', '88119', '88022', '88141', '88158', '88058', '88172', '88075', '88026', '88090', '88152', '88074', '88055', '88096', '88068', '88122', '88097', '88033', '88023', '88162', '88049', '88138', '88122', '88106', '88099', '88162', '88055', '88156']
file_ID

- We have 109 total examinations, split across 55 2D and 54 3D studies. After extracting unique IDs, we find 57 distinct patients, suggesting that many had both 2D and 3D imaging.
- 3 file IDs appear only in 2D studies and 2 appear only in 3D. The rest (52) had both imaging modalities.

### Comparing Spreadsheet Study IDs with Local Data

In this step, we compare the list of study IDs from the **JJ-labeled spreadsheet** with the **locally available ultrasound data** on disk.

- **Local 3D data**: `B:\US_Data_backup\Elbow Data\3D iU22 Elbow Data\_3D iU22 Elbow Data`
- **Local 2D data**: `B:\US_Data_backup\Elbow Data\2D Lumify Elbow Data\_2D Lumify Elbow Data`

We:
1. Extracted numeric study IDs from folder/file names.
2. Identified:
   -  IDs found in both spreadsheet and local storage
   -  IDs present in the spreadsheet but missing locally
   -  IDs present locally but not listed in the spreadsheet

This comparison ensures data consistency before further analysis and flags any missing or extra cases that need investigation.

In [10]:
# === Define local paths ===
folder_2D = Path(r'B:\US_Data_backup\Elbow Data\2D Lumify Elbow Data\_2D Lumify Elbow Data')
folder_3D = Path(r'B:\US_Data_backup\Elbow Data\3D iU22 Elbow Data\_3D iU22 Elbow Data')


def extract_study_ids(folder):
    ids = set()
    for f in folder.iterdir():
        # Match IDs like 88105 or W00004
        match = re.search(r'\b(?:W\d{5}|\d{5})\b', f.name, re.IGNORECASE)
        if match:
            ids.add(match.group().upper())  # Ensure consistent casing (W00004 vs w00004)
    return sorted(ids)

# Extract from local folders
local_2D_ids = extract_study_ids(folder_2D)
local_3D_ids = extract_study_ids(folder_3D)

# Convert spreadsheet IDs to same format (e.g., strings like '88105' or 'W00004')
spreadsheet_2D_ids = [str(i).upper().zfill(5) if isinstance(i, int) else str(i).upper() for i in IDs_2D]
spreadsheet_3D_ids = [str(i).upper().zfill(5) if isinstance(i, int) else str(i).upper() for i in IDs_3D]

# Convert to sets
spreadsheet_2D_ids = set(spreadsheet_2D_ids)
spreadsheet_3D_ids = set(spreadsheet_3D_ids)
local_2D_ids = set(local_2D_ids)
local_3D_ids = set(local_3D_ids)

# Compare
missing_2D = spreadsheet_2D_ids - local_2D_ids
extra_2D = local_2D_ids - spreadsheet_2D_ids
common_2D = spreadsheet_2D_ids & local_2D_ids

missing_3D = spreadsheet_3D_ids - local_3D_ids
extra_3D = local_3D_ids - spreadsheet_3D_ids
common_3D = spreadsheet_3D_ids & local_3D_ids

# Print summary
print("=== 2D Comparison ===")
print(" Common:", sorted(common_2D))
print(" Missing in local 2D:", sorted(missing_2D))
print(" Extra in local 2D:", sorted(extra_2D))

print("\n=== 3D Comparison ===")
print(" Common:", sorted(common_3D))
print(" Missing in local 3D:", sorted(missing_3D))
print(" Extra in local 3D:", sorted(extra_3D))

=== 2D Comparison ===
 Common: ['88010', '88015', '88017', '88022', '88023', '88025', '88026', '88033', '88034', '88036', '88037', '88049', '88055', '88058', '88068', '88069', '88072', '88074', '88075', '88077', '88078', '88082', '88084', '88090', '88091', '88094', '88095', '88096', '88097', '88098', '88099', '88105', '88106', '88111', '88112', '88114', '88118', '88119', '88122', '88131', '88132', '88133', '88135', '88138', '88141', '88145', '88148', '88150', '88152', '88154', '88156', '88158', '88162', '88170', '88172']
 Missing in local 2D: []
 Extra in local 2D: ['88019', '88028', '88057', '88061', '88070', '88102', '88126', '88128', '88137', '88159', 'W00001', 'W00008']

=== 3D Comparison ===
 Common: ['88010', '88015', '88017', '88022', '88023', '88025', '88026', '88033', '88034', '88037', '88042', '88055', '88058', '88068', '88069', '88072', '88074', '88075', '88077', '88078', '88082', '88084', '88090', '88091', '88093', '88094', '88095', '88096', '88098', '88099', '88105', '8810

- All spreadsheet-listed 2D and 3D study IDs are present locally. the extras here correspond to the dropped cases.

In [13]:
df['file_id'] = df['study_id'].str.extract(r'(W\d{5}|\d{5})', expand=False).str.upper()

# Step 2: Separate 2D and 3D entries
df_2D = df[df['study_id'].str.contains('2D', case=False)].copy()
df_3D = df[df['study_id'].str.contains('3D', case=False)].copy()

# Step 3: Find common subjects
common_ids = set(df_2D['file_id']) & set(df_3D['file_id'])

print(f"Found {len(common_ids)} studies with both 2D and 3D")

# Filter 2D and 3D rows to only common file_ids
df_2D_common = df_2D[df_2D['file_id'].isin(common_ids)]
df_3D_common = df_3D[df_3D['file_id'].isin(common_ids)]

# Count xr_fracture = 1 in each
fx_2D_common = df_2D_common[df_2D_common['xr_fracture'] == 1].shape[0]
fx_3D_common = df_3D_common[df_3D_common['xr_fracture'] == 1].shape[0]

normal_2D_common = df_2D_common[df_2D_common['xr_fracture'] == 0].shape[0]
normal_3D_common = df_3D_common[df_3D_common['xr_fracture'] == 0].shape[0]  

print("XR Fracture Counts for Common 2D/3D Studies:")
print(f"→ 2D view: {fx_2D_common} fractures", 
      f"({normal_2D_common} normals)")
print(f"→ 3D view: {fx_3D_common} fractures",   
      f"({normal_3D_common} normals)")


Found 52 studies with both 2D and 3D
XR Fracture Counts for Common 2D/3D Studies:
→ 2D view: 13 fractures (39 normals)
→ 3D view: 13 fractures (39 normals)



To check label consistency for subjects that have both 2D and 3D studies, you can:

1. Identify the intersection of 2D and 3D IDs.

2. For each overlapping study, compare the values in the label columns (e.g., us_effusion, xr_effusion, etc.).

3. Report any mismatches.

In [14]:
# Step 1: Ensure file_id is alphanumeric
df['file_id'] = df['study_id'].str.extract(r'(W\d{5}|\d{5})', expand=False).str.upper()

# Step 2: Separate 2D and 3D entries
df_2D = df[df['study_id'].str.contains('2D', case=False)].copy()
df_3D = df[df['study_id'].str.contains('3D', case=False)].copy()

# Step 3: Find common subjects
common_ids = set(df_2D['file_id']) & set(df_3D['file_id'])

print(f"Found {len(common_ids)} studies with both 2D and 3D")

# Step 4: Compare labels for each common subject
inconsistent_cases = []

for file_id in sorted(common_ids):
    row_2d = df_2D[df_2D['file_id'] == file_id].iloc[0]
    row_3d = df_3D[df_3D['file_id'] == file_id].iloc[0]

    diffs = {}
    for col in ['us_effusion', 'xr_effusion', 'us_fracture', 'xr_fracture']:
        if row_2d[col] != row_3d[col]:
            diffs[col] = (row_2d[col], row_3d[col])

    if diffs:
        inconsistent_cases.append({
            'file_id': file_id,
            'differences': diffs
        })

# Step 5: Print mismatches
if inconsistent_cases:
    print("Inconsistent labeling found for the following studies:")
    for case in inconsistent_cases:
        print(f"Study {case['file_id']}:")
        for label, (v2d, v3d) in case['differences'].items():
            print(f"  - {label}: 2D = {v2d}, 3D = {v3d}")
else:
    print("All common studies have consistent labeling between 2D and 3D")

print('Number of inconsistent cases:', len(inconsistent_cases))    


Found 52 studies with both 2D and 3D
Inconsistent labeling found for the following studies:
Study 88015:
  - us_fracture: 2D = 0, 3D = 1
Study 88022:
  - us_effusion: 2D = 1, 3D = 0
Study 88023:
  - us_effusion: 2D = 1, 3D = 0
  - us_fracture: 2D = 1, 3D = 0
Study 88025:
  - us_fracture: 2D = 1, 3D = 0
Study 88026:
  - us_fracture: 2D = 1, 3D = 0
Study 88034:
  - us_effusion: 2D = 1, 3D = 0
  - us_fracture: 2D = 1, 3D = 0
Study 88037:
  - us_fracture: 2D = 1, 3D = 0
Study 88068:
  - us_fracture: 2D = 1, 3D = 0
Study 88072:
  - us_fracture: 2D = 1, 3D = 0
Study 88074:
  - us_effusion: 2D = 1, 3D = 0
  - us_fracture: 2D = 1, 3D = 0
Study 88075:
  - us_effusion: 2D = 1, 3D = 0
Study 88099:
  - us_fracture: 2D = 1, 3D = 0
Study 88106:
  - us_fracture: 2D = 0, 3D = 1
Study 88132:
  - us_fracture: 2D = 1, 3D = 0
Study 88133:
  - us_fracture: 2D = 0, 3D = 1
Study 88138:
  - us_effusion: 2D = 1, 3D = 0
  - us_fracture: 2D = 1, 3D = 0
Study 88156:
  - us_effusion: 2D = 0, 3D = 1
  - us_fracture

- Of the 52 subjects with both 2D and 3D scans, 19 show label inconsistencies, primarily in us_fracture and us_effusion. 

In [37]:
file_2d_dor = nib.load(r"B:\US_Data_backup\Elbow Data\2D Lumify Elbow Data\_2D Lumify Elbow Data\88023 elbow 2D\88023-right-elbow-dor.nii.gz").get_fdata()
file_2d_vul = nib.load(r"B:\US_Data_backup\Elbow Data\2D Lumify Elbow Data\_2D Lumify Elbow Data\88023 elbow 2D\88023-right-elbow-vul.nii.gz").get_fdata()

file_3d_dor = nib.load(r"B:\US_Data_backup\Elbow Data\3D iU22 Elbow Data\_3D iU22 Elbow Data\88023 elbow 3D\88023-right-elbow-dor.nii.gz").get_fdata()
file_3d_vul = nib.load(r"B:\US_Data_backup\Elbow Data\3D iU22 Elbow Data\_3D iU22 Elbow Data\88023 elbow 3D\88023-right-elbow-vul.nii.gz").get_fdata()

print("2D dor shape:", file_2d_dor.shape)
print("2D vul shape:", file_2d_vul.shape)
print("3D dor shape:", file_3d_dor.shape)
print("3D vul shape:", file_3d_vul.shape)

print("2d dor labels:", np.unique(file_2d_dor))
print("2d vul labels:", np.unique(file_2d_vul))
print("3d dor labels:", np.unique(file_3d_dor))
print("3d vul labels:", np.unique(file_3d_vul))

2D dor shape: (1024, 768, 165)
2D vul shape: (1024, 768, 99)
3D dor shape: (800, 600, 127)
3D vul shape: (800, 600, 127)
2d dor labels: [0. 1. 2. 4. 6.]
2d vul labels: [0. 1. 4. 6.]
3d dor labels: [0. 1. 2. 4.]
3d vul labels: [0. 1. 4.]


- Of the 52 subjects with both 2D and 3D scans, 19 show label inconsistencies, primarily in us_fracture and us_effusion. 

## Diagnostic Performance Analysis

In this section, we evaluate the diagnostic accuracy of ultrasound (US) in detecting elbow **fractures** and **joint effusions**, using X-ray (XR) as the reference standard (ground truth). The analysis is limited to the **52 subjects** who underwent **both 2D and 3D ultrasound** imaging.

We assess performance using the following metrics:
- **Sensitivity (Recall)**: How well US detects XR-confirmed positives (fractures or effusions).
- **Specificity**: How well US avoids false positives when XR is normal.
- **Positive Predictive Value (PPV)**: Probability that a positive US finding is truly positive.
- **Negative Predictive Value (NPV)**: Probability that a negative US finding is truly negative.

The comparison is performed **separately** for:
- **2D US vs XR**
- **3D US vs XR**

The following evaluations are conducted:
- **Fracture detection using US compared to XR**
- **Effusion detection using US compared to XR**
- **Fracture detection using US detected effusion**



In [40]:
# Ensure `file_id` column is defined
df['file_id'] = df['study_id'].str.extract(r'(W\d{5}|\d{5})', expand=False).str.upper()

# Separate 2D and 3D entries
df_2D = df[df['study_id'].str.contains('2D', case=False)].copy()
df_3D = df[df['study_id'].str.contains('3D', case=False)].copy()

# Identify common IDs
common_ids = set(df_2D['file_id']) & set(df_3D['file_id'])

# Filter to just the 52 common subjects
df_2D_common = df_2D[df_2D['file_id'].isin(common_ids)].copy()
df_3D_common = df_3D[df_3D['file_id'].isin(common_ids)].copy()

In [41]:
def confusion_counts(pred, truth):
    """Return TP, FP, TN, FN counts given two equal-length 0/1 arrays."""
    tp = ((pred == 1) & (truth == 1)).sum()
    fp = ((pred == 1) & (truth == 0)).sum()
    tn = ((pred == 0) & (truth == 0)).sum()
    fn = ((pred == 0) & (truth == 1)).sum()
    return tp, fp, tn, fn

def metrics(tp, fp, tn, fn):
    sens = tp / (tp + fn) if (tp + fn) else float("nan")
    spec = tn / (tn + fp) if (tn + fp) else float("nan")
    ppv  = tp / (tp + fp) if (tp + fp) else float("nan")
    npv  = tn / (tn + fn) if (tn + fn) else float("nan")
    return sens, spec, ppv, npv

In [47]:
def show_results(name, df, pred_col, truth_col):
    tp, fp, tn, fn = confusion_counts(df[pred_col], df[truth_col])
    sens, spec, ppv, npv = metrics(tp, fp, tn, fn)
    total = len(df)
    print(f"\n=== {name} ===")
    print(f"Sample size: {total} cases")
    print(f"TP={tp}, FP={fp}, TN={tn}, FN={fn}")
    print(f"Sensitivity (recall): {sens:.3f}")
    print(f"Specificity:          {spec:.3f}")
    print(f"PPV (precision):      {ppv:.3f}")
    print(f"NPV:                  {npv:.3f}")


In [49]:
# A. Effusion (US) vs Effusion (XR) on 2D
show_results("2D US Effusion vs XR Effusion", df_2D_common, "us_effusion", "xr_effusion")
# Effusion (US) vs Effusion (XR) on 3D
show_results("3D US Effusion vs XR Effusion", df_3D_common, "us_effusion", "xr_effusion")


=== 2D US Effusion vs XR Effusion ===
Sample size: 52 cases
TP=24, FP=8, TN=15, FN=5
Sensitivity (recall): 0.828
Specificity:          0.652
PPV (precision):      0.750
NPV:                  0.750

=== 3D US Effusion vs XR Effusion ===
Sample size: 52 cases
TP=22, FP=3, TN=20, FN=7
Sensitivity (recall): 0.759
Specificity:          0.870
PPV (precision):      0.880
NPV:                  0.741


In [50]:
# B. Fracture (US) vs Fracture (XR) on 2D
show_results("2D US Fracture vs XR Fracture", df_2D_common, "us_fracture", "xr_fracture")

# Evaluate fracture detection (US vs XR) on 3D
show_results("3D US Fracture vs XR Fracture", df_3D_common, "us_fracture", "xr_fracture")



=== 2D US Fracture vs XR Fracture ===
Sample size: 52 cases
TP=10, FP=13, TN=26, FN=3
Sensitivity (recall): 0.769
Specificity:          0.667
PPV (precision):      0.435
NPV:                  0.897

=== 3D US Fracture vs XR Fracture ===
Sample size: 52 cases
TP=9, FP=6, TN=33, FN=4
Sensitivity (recall): 0.692
Specificity:          0.846
PPV (precision):      0.600
NPV:                  0.892


In [68]:
# C. Effusion (US) predicting Fracture (XR) on 2D
show_results("2D US Effusion vs XR Fracture", df_2D_common, "us_effusion", "xr_fracture")
# Effusion (US) predicting Fracture (XR) on 3D
show_results("3D US Effusion vs XR Fracture", df_3D_common, "us_effusion", "xr_fracture")



=== 2D US Effusion vs XR Fracture ===
Sample size: 52 cases
TP=12, FP=20, TN=19, FN=1
Sensitivity (recall): 0.923
Specificity:          0.487
PPV (precision):      0.375
NPV:                  0.950

=== 3D US Effusion vs XR Fracture ===
Sample size: 52 cases
TP=12, FP=13, TN=26, FN=1
Sensitivity (recall): 0.923
Specificity:          0.667
PPV (precision):      0.480
NPV:                  0.963


In [24]:
import os
import nibabel as nib
import numpy as np
from pathlib import Path

def get_all_labels_in_folder(folder_path, include_subfolders=True):
    folder = Path(folder_path)
    ext = [".nii", ".nii.gz"]
    all_labels = set()

    if include_subfolders:
        nifti_files = list(folder.rglob("*"))  # recursive
    else:
        nifti_files = list(folder.glob("*"))   # only top level

    nifti_files = [f for f in nifti_files if f.suffix in ext or f.suffixes == [".nii", ".gz"]]

    print(f"Found {len(nifti_files)} NIfTI files.\n")

    for f in nifti_files:
        try:
            img = nib.load(str(f))
            data = img.get_fdata()
            labels = np.unique(data)
            labels = labels.astype(int) if np.all(labels == labels.astype(int)) else labels
            print(f"{f.name}: Labels = {labels}")
            all_labels.update(labels.tolist())
        except Exception as e:
            print(f"Error loading {f.name}: {e}")

    print("\n=== All unique labels across folder ===")
    print(sorted(all_labels))

In [25]:
get_all_labels_in_folder(r"M:\ElbowProject\data\_2D Lumify Elbow Data\masks", include_subfolders=True)

Found 130 NIfTI files.

88010-left-elbow-dor.nii.gz: Labels = [0 1 2]
88010-left-elbow-vul.nii.gz: Labels = [0 1 4]
88015-left-elbow-dor.nii.gz: Labels = [0 1 2 4 6]
88015-left-elbow-vul.nii.gz: Labels = [0 1 6]
88017-right-elbow-dor.nii.gz: Labels = [0 1]
88017-right-elbow-dor2.nii.gz: Labels = [0 1 2 4]
88017-right-elbow-vul.nii.gz: Labels = [0 1 3 4]
88019-right-elbow-dor.nii.gz: Labels = [0 1 2 4 6]
88019-right-elbow-vul.nii.gz: Labels = [0 1 4 6]
88022-left-elbow-dor.nii.gz: Labels = [0 1 4 6]
88022-left-elbow-vul.nii.gz: Labels = [0 1 2 3 4]
88023-right-elbow-dor.nii.gz: Labels = [0 1 2 4 6]
88023-right-elbow-vul.nii.gz: Labels = [0 1 4 6]
88025-right-elbow-vul.nii.gz: Labels = [0 1 2 3 4 6]
88026-right-elbow-dor.nii.gz: Labels = [0 1]
88026-right-elbow-vul.nii.gz: Labels = [0 1 4 6]
88028-left-elbow-dor.nii.gz: Labels = [0 1 2 4 6]
88028-left-elbow-vul.nii.gz: Labels = [0 1 2 3 4 6]
88033-left-elbow-dor.nii.gz: Labels = [0 1 2 4]
88033-left-elbow-vul.nii.gz: Labels = [0 1 2 3 4]

In [53]:
import nibabel as nib
import numpy as np
from pathlib import Path

def get_files_with_label_7(mask_folder):
    mask_folder = Path(mask_folder)
    files_with_label_7 = []

    for nii_file in mask_folder.glob("*.nii*"):
        try:
            mask = nib.load(nii_file).get_fdata().astype(np.uint8)
            if 7 in np.unique(mask):
                files_with_label_7.append(nii_file.name)
        except Exception as e:
            print(f"⚠️ Error reading {nii_file.name}: {e}")

    return files_with_label_7
fracture_cases = get_files_with_label_7(
    r"M:\ElbowProject\data\_2D Lumify Elbow Data\masks"
)

print(f"🦴 Found {len(fracture_cases)} files with label 7:")
for f in fracture_cases:
    print(f" - {f}")


🦴 Found 12 files with label 7:
 - 88075-right-elbow-dor.nii.gz
 - 88090-left-elbow-dor.nii.gz
 - 88094-right-elbow-dor.nii.gz
 - 88094-right-elbow-vul.nii.gz
 - 88098-left-elbow-dor.nii.gz
 - 88098-left-elbow-vul.nii.gz
 - 88114-right-elbow-dor.nii.gz
 - 88114-right-elbow-vul.nii.gz
 - 88135-right-elbow-vul.nii.gz
 - 88145-right-elbow-dor.nii.gz
 - 88148-right-elbow-dor.nii.gz
 - 88150-right-elbow-vul.nii.gz


In [55]:
# Extract unique IDs
ids = sorted({re.match(r'^\d+', name).group() for name in fracture_cases})
print(ids)

['88075', '88090', '88094', '88098', '88114', '88135', '88145', '88148', '88150']


In [57]:
# Filter rows with US fracture = 1
df_2D_fx = df_2D_common[df_2D_common['us_fracture'] == 1]

# Extract unique alphanumeric file IDs
fx_ids_2D = sorted(df_2D_fx['file_id'].unique())

print(fx_ids_2D)

['88023', '88025', '88026', '88034', '88037', '88068', '88072', '88074', '88090', '88094', '88098', '88099', '88114', '88118', '88119', '88122', '88132', '88138', '88145', '88148', '88150', '88152', '88162']


In [61]:
only_in_list1 = sorted(set(ids) - set(fx_ids_2D))
only_in_list2 = sorted(set(fx_ids_2D) - set(ids))

print("fracture labeled on the masks but labeled no fracture in the spread sheet:", only_in_list1)
print("fracture in the spread sheet  but has no fracture label (7) in the segmentation mask", only_in_list2)

fracture labeled on the masks but labeled no fracture in the spread sheet: ['88075', '88135']
fracture in the spread sheet  but has no fracture label (7) in the segmentation mask ['88023', '88025', '88026', '88034', '88037', '88068', '88072', '88074', '88099', '88118', '88119', '88122', '88132', '88138', '88152', '88162']


In [60]:
import nibabel as nib
import numpy as np
from pathlib import Path

def get_files_with_label_6(mask_folder):
    mask_folder = Path(mask_folder)
    files_with_label_6 = []

    for nii_file in mask_folder.glob("*.nii*"):
        try:
            mask = nib.load(nii_file).get_fdata().astype(np.uint8)
            if 6 in np.unique(mask):
                files_with_label_6.append(nii_file.name)
        except Exception as e:
            print(f"⚠️ Error reading {nii_file.name}: {e}")

    return files_with_label_6
effusion_cases = get_files_with_label_6(
    r"M:\ElbowProject\data\_2D Lumify Elbow Data\masks"
)

print(f"🦴 Found {len(effusion_cases)} files with label 6:")
for f in effusion_cases:
    print(f" - {f}")


🦴 Found 74 files with label 6:
 - 88015-left-elbow-dor.nii.gz
 - 88015-left-elbow-vul.nii.gz
 - 88019-right-elbow-dor.nii.gz
 - 88019-right-elbow-vul.nii.gz
 - 88022-left-elbow-dor.nii.gz
 - 88023-right-elbow-dor.nii.gz
 - 88023-right-elbow-vul.nii.gz
 - 88025-right-elbow-vul.nii.gz
 - 88026-right-elbow-vul.nii.gz
 - 88028-left-elbow-dor.nii.gz
 - 88028-left-elbow-vul.nii.gz
 - 88036-left-elbow-dor.nii.gz
 - 88036-left-elbow-vul.nii.gz
 - 88037-left-elbow-dor.nii.gz
 - 88069-left-elbow-dor.nii.gz
 - 88069-left-elbow-vul.nii.gz
 - 88072-left-elbow-dor.nii.gz
 - 88072-left-elbow-vul.nii.gz
 - 88074-left-elbow-dor.nii.gz
 - 88075-right-elbow-vul.nii.gz
 - 88078-left-elbow-dor.nii.gz
 - 88078-left-elbow-vul.nii.gz
 - 88082-right-elbow-vul.nii.gz
 - 88084-left-elbow-dor.nii.gz
 - 88090-left-elbow-dor.nii.gz
 - 88090-left-elbow-vul.nii.gz
 - 88091-right-elbow-dor.nii.gz
 - 88094-right-elbow-dor.nii.gz
 - 88094-right-elbow-vul.nii.gz
 - 88097-right-elbow-dor.nii.gz
 - 88097-right-elbow-vul.ni

In [64]:
effusion_cases

['88015-left-elbow-dor.nii.gz',
 '88015-left-elbow-vul.nii.gz',
 '88019-right-elbow-dor.nii.gz',
 '88019-right-elbow-vul.nii.gz',
 '88022-left-elbow-dor.nii.gz',
 '88023-right-elbow-dor.nii.gz',
 '88023-right-elbow-vul.nii.gz',
 '88025-right-elbow-vul.nii.gz',
 '88026-right-elbow-vul.nii.gz',
 '88028-left-elbow-dor.nii.gz',
 '88028-left-elbow-vul.nii.gz',
 '88036-left-elbow-dor.nii.gz',
 '88036-left-elbow-vul.nii.gz',
 '88037-left-elbow-dor.nii.gz',
 '88069-left-elbow-dor.nii.gz',
 '88069-left-elbow-vul.nii.gz',
 '88072-left-elbow-dor.nii.gz',
 '88072-left-elbow-vul.nii.gz',
 '88074-left-elbow-dor.nii.gz',
 '88075-right-elbow-vul.nii.gz',
 '88078-left-elbow-dor.nii.gz',
 '88078-left-elbow-vul.nii.gz',
 '88082-right-elbow-vul.nii.gz',
 '88084-left-elbow-dor.nii.gz',
 '88090-left-elbow-dor.nii.gz',
 '88090-left-elbow-vul.nii.gz',
 '88091-right-elbow-dor.nii.gz',
 '88094-right-elbow-dor.nii.gz',
 '88094-right-elbow-vul.nii.gz',
 '88097-right-elbow-dor.nii.gz',
 '88097-right-elbow-vul.nii.

In [65]:
# Extract unique alphanumeric IDs (e.g., W00008 or 88075)
ids_effusion = sorted({re.match(r'^(W\d{5}|\d{5})', name).group().upper() for name in effusion_cases})
print(ids_effusion)

['88015', '88019', '88022', '88023', '88025', '88026', '88028', '88036', '88037', '88069', '88072', '88074', '88075', '88078', '88082', '88084', '88090', '88091', '88094', '88097', '88098', '88099', '88106', '88111', '88114', '88118', '88119', '88122', '88126', '88131', '88132', '88135', '88137', '88141', '88145', '88148', '88150', '88152', '88154', '88156', '88158', '88162', 'W00001', 'W00008']


In [66]:
# Filter rows with US fracture = 1
df_2D_effusion = df_2D_common[df_2D_common['us_effusion'] == 1]

# Extract unique alphanumeric file IDs
df_2D_effusion = sorted(df_2D_effusion['file_id'].unique())

print(df_2D_effusion)

['88015', '88022', '88023', '88025', '88026', '88033', '88034', '88037', '88072', '88074', '88075', '88078', '88090', '88091', '88094', '88098', '88099', '88106', '88114', '88118', '88119', '88122', '88132', '88135', '88138', '88141', '88145', '88150', '88152', '88154', '88158', '88162']


In [67]:
only_in_list1 = sorted(set(ids_effusion) - set(df_2D_effusion))
only_in_list2 = sorted(set(df_2D_effusion) - set(ids_effusion))

print("effusion labeled on the masks but labeled no effusion in the spread sheet:", only_in_list1)
print("effusion in the spread sheet  but has no effusion label (6) in the segmentation mask", only_in_list2)

effusion labeled on the masks but labeled no effusion in the spread sheet: ['88019', '88028', '88036', '88069', '88082', '88084', '88097', '88111', '88126', '88131', '88137', '88148', '88156', 'W00001', 'W00008']
effusion in the spread sheet  but has no effusion label (6) in the segmentation mask ['88033', '88034', '88138']
